In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite
/kaggle/input/glove-6b-100-d/glove.6B.100d.txt


In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv(r"/kaggle/input/amazon-fine-food-reviews/Reviews.csv")

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [5]:
dataset.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
print(dataset[['Score','Text']].head(5))

   Score                                               Text
0      5  I have bought several of the Vitality canned d...
1      1  Product arrived labeled as Jumbo Salted Peanut...
2      4  This is a confection that has been around a fe...
3      2  If you are looking for the secret ingredient i...
4      5  Great taffy at a great price.  There was a wid...


In [7]:
lmtpData=dataset[['Text','Summary','Score']].sample(n=1000)
print(lmtpData)

                                                     Text  \
135674  These are not original cliff bars, but the sma...   
16838   I don't often drink soda for a couple reasons....   
109996  I love the coffee and I love the transaction. ...   
397986  I don't usually have to cook for myself becaus...   
87293   This stuff is tasty but caveat emptor: it's gl...   
...                                                   ...   
306239  I've used this for years and it's the best I'v...   
483010  This is heavenly in espresso! One pump of this...   
358824  I have tried many flavored coconut waters and ...   
65456   I love nuts as a snack. They're good for you, ...   
385942  I bought this same brand at a local health foo...   

                                 Summary  Score  
135674       expensive cliff crunch bars      1  
16838                        Very tastey      5  
109996  Great transaction! Great Coffee!      5  
397986                           not bad      4  
87293   4-star ta

### ADDING SENTIMENT COLUMN

In [8]:
#0-negative, 1-positive
def lblScore(score):
    if score<=2:
        return 0
    elif score>=4:
        return 1
    else:
        return 2
lmtpData['Sentiment']=lmtpData['Score'].apply(lblScore)
lmtpData.head(5)

,Text,Summary,Score,Sentiment
135674,"These are not original cliff bars, but the sma...",expensive cliff crunch bars,1,0
16838,I don't often drink soda for a couple reasons....,Very tastey,5,1
109996,I love the coffee and I love the transaction. ...,Great transaction! Great Coffee!,5,1
397986,I don't usually have to cook for myself becaus...,not bad,4,1
87293,This stuff is tasty but caveat emptor: it's gl...,"4-star taste, 2-star ingredients",2,0


In [9]:
lmtpData['Sentiment'].value_counts()

Sentiment
1    783
0    151
2     66
Name: count, dtype: int64

#### VECTORS

In [10]:
import numpy as np
glove_path="/kaggle/input/glove-6b-100-d/glove.6B.100d.txt"
def loadGloveEmbeddings(filePath,embedding_dim=100):
    embeddingsIndex={}
    with open(filePath,encoding="utf8") as f:
        for line in f:
            values=line.split()
            word=values[0]
            coefs=np.asarray(values[1:],dtype='float32')
            embeddingsIndex[word]=coefs
        return embeddingsIndex

        

In [11]:
gloveEmbeddings=loadGloveEmbeddings(glove_path)
print("Loaded {} word vectors.".format(len(gloveEmbeddings)))

Loaded 400000 word vectors.


#### APPLYING SMOTE